In [1]:
import pandas as pd
import json
import os

In [2]:
frames = []
for root, dirs, files in os.walk("../../results/final_dataset_train"):
   for name in files:
      if name.endswith(".json"):
          frame = pd.read_json(os.path.join(root, name), orient="record", lines=True)
          frames.append(frame)
results = pd.concat(frames)
results.columns

Index([&#39;task_id&#39;, &#39;input_src_path&#39;, &#39;max_vocab_size&#39;, &#39;problem_type&#39;,
       &#39;window_size&#39;, &#39;step_size&#39;, &#39;encode_type&#39;, &#39;test_split_percentage&#39;,
       &#39;train_dev_split_percentage&#39;, &#39;oversampling_enabled&#39;,
       &#39;ratio_after_oversampling&#39;, &#39;undersampling_enabled&#39;,
       &#39;ratio_after_undersampling&#39;, &#39;embedding_vecor_length&#39;, &#39;epochs&#39;,
       &#39;batch_size&#39;, &#39;num_lstm_cells&#39;, &#39;dropout_emb_lstm&#39;,
       &#39;dropout_lstm_dense&#39;, &#39;recall&#39;, &#39;precision&#39;, &#39;roc&#39;, &#39;f1&#39;,
       &#39;train_recall&#39;, &#39;train_precision&#39;, &#39;train_roc&#39;, &#39;train_f1&#39;,
       &#39;train_dev_recall&#39;, &#39;train_dev_precision&#39;, &#39;train_dev_roc&#39;,
       &#39;train_dev_f1&#39;, &#39;cm&#39;, &#39;cm_normalized&#39;, &#39;n_trees_in_forest&#39;,
       &#39;max_features&#39;, &#39;n_estimators&#39;, &#39;learn

In [3]:
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 4)


In [4]:
results["ratio_after_oversampling"][results.oversampling_enabled == False] = "-"
results["ratio_after_undersampling"][results.undersampling_enabled == False] = "-"
results

,task_id,input_src_path,max_vocab_size,problem_type,window_size,step_size,encode_type,test_split_percentage,train_dev_split_percentage,oversampling_enabled,ratio_after_oversampling,undersampling_enabled,ratio_after_undersampling,embedding_vecor_length,epochs,batch_size,num_lstm_cells,dropout_emb_lstm,dropout_lstm_dense,recall,precision,roc,f1,train_recall,train_precision,train_roc,train_f1,train_dev_recall,train_dev_precision,train_dev_roc,train_dev_f1,cm,cm_normalized,n_trees_in_forest,max_features,n_estimators,learning_rate,subsample,class_weight
0,TaskEvaluateLstm_256_0_2_0_2_7c4c8888a2,final_dataset,100000,CONDITION_COMPARISON,20,3,False,0.2,0.1,False,-,False,-,32.0,2.0,256.0,10.0,0.2,0.2,0.9907,0.9698,0.9999,0.9801,0.9947,0.9782,1.0000,0.9864,0.9899,0.9688,0.9999,0.9792,"[[1031334, 1631], [493, 52342]]","[[0.9984210500839811, 0.001578949916018], [0.0...",NaN,NaN,NaN,NaN,NaN,NaN
0,TaskEvaluateRandomForest_True_final_dataset_sq...,final_dataset,1000,RETURN_NULL,20,3,True,0.2,0.1,True,0.5,False,-,NaN,NaN,NaN,NaN,NaN,NaN,0.7313,0.9859,0.9992,0.8397,1.0000,1.0000,1.0000,1.0000,0.7270,0.9952,0.9994,0.8402,"[[1083382, 25], [643, 1750]]","[[0.999976924646047, 2.3075353952854282e-05], ...",100.0,sqrt,NaN,NaN,NaN,NaN
0,TaskEvaluateGradientBoostingClassifier_True_fi...,final_dataset,1000,RETURN_NULL,20,3,True,0.2,0.1,False,-,True,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.9883,0.1289,0.9986,0.2281,0.9917,0.9793,0.9992,0.9855,0.9887,0.1236,0.9988,0.2197,"[[1067424, 15983], [28, 2365]]","[[0.9852474647108611, 0.014752535289138], [0.0...",NaN,NaN,100.0,0.2,1.0,NaN
0,TaskEvaluateGradientBoostingClassifier_True_se...,second_large_dataset,100000,CONDITION_COMPARISON,20,3,True,0.2,0.1,False,-,True,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.9727,0.6734,0.9954,0.7958,0.9736,0.9544,0.9956,0.9639,0.9724,0.6722,0.9954,0.7949,"[[1013046, 25016], [1450, 51575]]","[[0.97590124674634, 0.024098753253659], [0.027...",NaN,NaN,200.0,0.2,1.0,NaN
0,TaskEvaluateLstm_64_0_2_0_2_fbd91344a9,final_dataset,100000,CONDITION_COMPARISON,20,3,False,0.2,0.1,False,-,True,0.5,32.0,3.0,64.0,10.0,0.2,0.2,0.9978,0.9436,0.9998,0.9699,0.9988,0.9968,1.0000,0.9978,0.9968,0.9424,0.9998,0.9688,"[[1029816, 3149], [118, 52717]]","[[0.99695149400028, 0.0030485059997190003], [0...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,TaskEvaluateRandomForest_True_final_dataset_sq...,final_dataset,100000,RETURN_NULL,20,3,True,0.2,0.1,False,-,True,0.1,NaN,NaN,NaN,NaN,NaN,NaN,0.9014,0.4905,0.9989,0.6353,1.0000,1.0000,1.0000,1.0000,0.8965,0.4861,0.9990,0.6304,"[[1081166, 2241], [236, 2157]]","[[0.9979315252716661, 0.002068474728333], [0.0...",100.0,sqrt,NaN,NaN,NaN,NaN
0,TaskEvaluateRandomForest_balanced_True_final_d...,final_dataset,100000,CONDITION_COMPARISON,20,3,True,0.2,0.1,True,0.5,False,-,NaN,NaN,NaN,NaN,NaN,NaN,0.8958,0.9749,0.9987,0.9337,1.0000,1.0000,1.0000,1.0000,0.8954,0.9738,0.9987,0.9330,"[[1031747, 1218], [5503, 47332]]","[[0.998820870019797, 0.0011791299802020002], [...",100.0,sqrt,NaN,NaN,NaN,balanced
0,TaskEvaluateLstm_256_0_2_0_2_f03085d3d0,final_dataset,100000,CONDITION_COMPARISON_SIMPLE,20,3,False,0.2,0.1,False,-,False,-,32.0,2.0,256.0,10.0,0.2,0.2,0.9738,0.9878,0.9999,0.9808,0.9833,0.9948,1.0000,0.9890,0.9746,0.9881,0.9999,0.9813,"[[1056699, 345], [754, 28002]]","[[0.9996736181275331, 0.00032638187246600004],...",NaN,NaN,NaN,NaN,NaN,NaN
0,TaskEvaluateRandomForest_True_final_dataset_sq...,final_dataset,1000,RETURN_NULL,20,3,True,0.2,0.1,True,1,False,-,NaN,NaN,NaN,NaN,NaN,NaN,0.7204,0.9857,0.9979,0.8324,1.0000,1.0000,1.0000,1.0000,0.7200,0.9928,0.9989,0.8347,"[[1083382, 25], [669, 1724]]","[[0.999976924646047, 2.3075353952854282e-05], ...",100.0,sqrt,NaN,NaN,NaN,NaN


In [5]:
#important fix, check later if all experiments rerun
results = results[results["max_vocab_size"] == 100000]
results = results[results["input_src_path"] == "final_dataset"]

In [6]:
merge_columns = ['window_size', 'step_size', 'encode_type','oversampling_enabled', 'ratio_after_oversampling', 'undersampling_enabled', 'ratio_after_undersampling', 'n_trees_in_forest', 'max_features', 'class_weight']

random_forest_RN = results[(results["problem_type"] == "RETURN_NULL")  & (results["n_trees_in_forest"].notnull())]
random_forest_RN.columns = random_forest_RN.columns.map(lambda a: a + "_RF" if a not in merge_columns else a)

random_forest_CC = results[(results["problem_type"] == "RETURN_NULL")  & (results["n_trees_in_forest"].notnull())]
random_forest_CC.columns = random_forest_CC.columns.map(lambda a: a + "_CC" if a not in merge_columns else a)

random_forest_CCS = results[(results["problem_type"] == "RETURN_NULL")  & (results["n_trees_in_forest"].notnull())]
random_forest_CCS.columns = random_forest_CCS.columns.map(lambda a: a + "_CCS" if a not in merge_columns else a)


merged = random_forest_RN.merge(random_forest_CC, on=merge_columns, suffixes=("_LEFT1", "_RIGHT1")).merge(random_forest_CCS, on=merge_columns, suffixes=("_LEFT2", "_RIGHT2"))

merged.columns

Index([&#39;task_id_RF&#39;, &#39;input_src_path_RF&#39;, &#39;max_vocab_size_RF&#39;,
       &#39;problem_type_RF&#39;, &#39;window_size&#39;, &#39;step_size&#39;, &#39;encode_type&#39;,
       &#39;test_split_percentage_RF&#39;, &#39;train_dev_split_percentage_RF&#39;,
       &#39;oversampling_enabled&#39;, &#39;ratio_after_oversampling&#39;,
       &#39;undersampling_enabled&#39;, &#39;ratio_after_undersampling&#39;,
       &#39;embedding_vecor_length_RF&#39;, &#39;epochs_RF&#39;, &#39;batch_size_RF&#39;,
       &#39;num_lstm_cells_RF&#39;, &#39;dropout_emb_lstm_RF&#39;, &#39;dropout_lstm_dense_RF&#39;,
       &#39;recall_RF&#39;, &#39;precision_RF&#39;, &#39;roc_RF&#39;, &#39;f1_RF&#39;, &#39;train_recall_RF&#39;,
       &#39;train_precision_RF&#39;, &#39;train_roc_RF&#39;, &#39;train_f1_RF&#39;,
       &#39;train_dev_recall_RF&#39;, &#39;train_dev_precision_RF&#39;, &#39;train_dev_roc_RF&#39;,
       &#39;train_dev_f1_RF&#39;, &#39;cm_RF&#39;, &#39;cm_normalized_RF&#39;, &#39;n_tr

In [7]:
import os
import jinja2
from jinja2 import Template
latex_jinja_env = jinja2.Environment(
    block_start_string='\BLOCK{',
    block_end_string='}',
    variable_start_string='\VAR{',
    variable_end_string='}',
    comment_start_string='\#{',
    comment_end_string='}',
    line_statement_prefix='%%',
    line_comment_prefix='%#',
    trim_blocks=True,
    autoescape=False)

In [8]:
with open("table_train_all.tex.jinja2", "r") as f:
    t = latex_jinja_env.from_string(f.read())
    rendered = t.render(data=merged.T.to_dict().values())
    print(rendered)

% Please add the following required packages to your document preamble:
% \usepackage{booktabs}
\begin{table}[]
\begin{tabular}{@{}llllllllllllll@{}}
\toprule
          &amp; \multicolumn{4}{c}{Parameter}                                                                                                                                                                                                                                   &amp; \multicolumn{3}{c}{RN}                                                                                                                                                                         &amp; \multicolumn{3}{c}{CCS}                                                                                                                                                                           &amp; \multicolumn{3}{c}{CC}                                                                                                                                                